In [2]:
import sys
!{sys.executable} -m pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/d3/b8/6798a0f91e595c0704a94c1e32a00ca930f77cb8f0f96c7e4dad4f9db1fe/mysql_connector_python-8.1.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     ---- -------------------------------- 61.4/527.0 kB 825.8 kB/s eta 0:00:01
     -------------------------------------  522.2/527.0 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.8 MB 20.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/10.8 MB 20.4 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/10.8 MB 20.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.8 MB 21.0 MB/s eta 0

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql

conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()

#mycursor.execute("CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")
#mycursor.execute("DROP TABLE customers")
conn.close()